#  R-CNN
入力画像に対して、物体が写っている領域の候補(region proposal)を約2000個抽出して、CNNのインプットの大きさに合うようにそれぞれの領域中の画像をリサイズして、それぞれの領域に対して、CNN
で特徴量を計算し、分類する。
領域分けとしては、selective searchを用いている
R-CNNにおいて分類がうまくいったとはという定義については、物体名と、領域の位置・大きさがあって初めて正解となる。
まとめると、入力画像に対して、領域の候補をSelective Searchで約２０００個抽出し、CNNのインプットの大きさに合うようにそれぞれの領域中の画像をリサイズし、それぞれの領域中の画像をリサイズして、それぞれの領域に対してCNNで特徴量を計算して、それぞれの領域で何が写っているかをSVMで分類する。
RCNNの欠点としては、領域候補2000個に対して、それぞれCNNを走らせているので、実行時間が遅い。
領域一致の指標 *IoU*

#  Fast R-CNN
あらかじめ特徴抽出用のCNNにより画像全体の特徴マップを作成して、物体候補の領域に対応する特徴マップをRoIPoolingによって切り出すことで特徴抽出を高速化し、特徴マップを作成するCNNにもbackprop可能

# Faster R-CNN
物体候補の領域の抽出にCNNベースのRegion Proposal Network (RPN) を利用することで、物体候補領域の高精度化を実現しつつ、真のend-to-end学習を実現.

CNNベースの物体検出器は物体候補領域をあらかじめ求めないといけない。
そのため、物体候補領域検出処理もCNNで行い、Region ProposalNetworkを提案してFast R-CNNのSSを置き換える。


# Region Proposal Network
![](https://www.slideshare.net/takashiabe338/fast-rcnnfaster-rcnn)
任意のサイズの画像を入力して、物体候補領域とそのスコアを出力する。
FastRCNNと同様にconv層を何段かかけて、feature mapを計算
Feature　map上で3*3の検出窓を走査、物体の有無を窓ごとに分類。
単純に考えると、スケール・アスペクト比一定の荒いsliding-windowをしているだけ。
→多様な形の領域候補を出せない
アンカーを導入して、window一つから複数領域のobjectnessを求める。
Anchors　
・window毎にk個のアンカーのobject/background分類
・4k個の回帰を行う
Anchorは３スケールでアスペクト比1:1,1:2,2:1の３種類で決め打ち


# 物体検出の指標
# mAP
ある画像の情報が与えられた時点までの適合率(Precision)の平均であるAP(AveragePrecision)の平均。APとはm個の正解ラベルが現れた時点で、モデルがm個のラベルの内、どれだけのラベルを検出できているかを平均的に表現したもの。全ての時点iに置けるAPを平均することで、より一般的な平均値を求めるためのもの。
# fps
比較的データの大きくなりがちな画像を、機械学習で扱うとなれば、処理時間が膨大となる
危険回避のために処理速度が求められる場合、処理速度の高速化は必要。
fpsは1秒あたりに何枚の画像を処理できるかという指標。

# IoU
実際に本物(TP+FN)である、あるいはポジティブ(TP+NP)であると予想した領域(TP+FP+FN)の内、モデルが正確に本物である認識した領域(TP)の割合を表す。

In [ ]:
https://qiita.com/yu4u/items/5cbe9db166a5d72f9eb8

# (1) 物体検出の分野にはどういった手法が存在したか。
#  R-CNN
入力画像に対して、領域の候補をSelective Searchで約２０００個抽出し、CNNのインプットの大きさに合うようにそれぞれの領域中の画像をリサイズし、それぞれの領域中の画像をリサイズして、それぞれの領域に対してCNNで特徴量を計算して、それぞれの領域で何が写っているかをSVMで分類する。
RCNNの欠点としては、領域候補2000個に対して、それぞれCNNを走らせているので、実行時間が遅い。
#  Fast R-CNN
あらかじめ特徴抽出用のCNNにより画像全体の特徴マップを作成して、物体候補の領域に対応する特徴マップをRoIPoolingによって切り出すことで特徴抽出を高速化し、特徴マップを作成するCNNにもbackprop可能。
# (2) Fasterとあるが、どういった仕組みで高速化したのか。
Conclusionの部分から
我々は、効率的で正確な地域提案の生成のためのRPNを提示した。畳み込みを共有する
下流の検出ネットワークを使用することで、地域提案のステップにはほとんどコストがかかりません。この方法は、統合されたディープ学習ベースのオブジェクト検出システムをほぼリアルタイムのフレームレートで実行することを可能にします。学習されたRPNはまた、地域の提案品質を改善し、したがって全体的な物体検出精度を向上させる。

# (3) One-Stageの手法とTwo-Stageの手法はどう違うのか。
Detectionのアルゴリズムには大きく２つの方式があり、
One-Stage MethodとTwo-stage-Method　がある。
One-stageは１段のCNNで直接bounding-box　の座標とクラスを推定する。
精度は劣るが検出速度は早い
Two-stageは２段階のプロセスで物体検出をする手法、①候補領域の抽出　②　クラスbouding boxの座標の推定。精度は高いが、遅い。

#  (4) RPNとは何か。
Region Proposal Networkは画像内から物体領域の候補となる箇所を抽出する。
元々はFast RCNNではこの物体領域抽出にSelective Searchと呼ばれる手法を使っていたため、
物体候補領域の抽出に時間がかかっていた。その箇所すらもEnd-to-Endで実施することで従来より高速・精度改善をしている。
Backboneの出力がRPNに対して、アンカーと呼ばれる物体らしい領域の候補を選定するための基となる領域を定義して、アンカーのそれぞれに対して、正解とするデータを定義する。そのあとアンカーに対してどれだけ、物体と皆すか（clf）とオフセット（reg,正解とする領域からどれだけずれているか）を得る。正解のデータを元にlossを得る。①その領域は物体か背景であるか、②どこに領域があるかの領域の位置（詳細）を計算。


#  (5) RoIプーリングとは何か。
https://arxiv.org/pdf/1504.08083.pdf 2.1The RoI pooling Layer
ある程度畳み込み処理を行ったfeature mapから、region proposalにあたる部分領域をうまく「固定サイズのfeature map」として抽出する。
元画像のregion proposalの領域を、feature map上に投影してみると、当然feature mapとサブピクセルレベルのずれが発生します。
RoI Poolingでは、このずれを丸め込みながらPoolingを行う。具体的には、region proposalの座標を整数値に丸めて、外接の長方形を得る
の後、その外接矩形をfeature mapのサイズと同じビン数（ここでは3x3）に等分割します。そして、feature mapのRoI内のピクセルを、それぞれ3x3つのビンのいずれかに割り当て、割り当てられたビンの中でmaxやaverageを取ることで、RoI Poolingの出力を得る。



#  (6) Anchorのサイズはどうするのが適切か。
それぞれの領域における最大の提案可能proposal数をk(ここではk=9)とするので、box数kに対しreg層の出力は4k、cls層の出力は2kとなる。
このk boxesはanchorsと呼ばれ、スケールやアスペクト比の種類数と繋がっている。
k=9では、スケール3種、アスペクト比3種であり、特徴マップのサイズがW*Hならアンカー数はWHkとなる(スライド幅1より)。
この文献においてスケールは128*128、256*256、512*512でアスペクト比は1/1、2/1、1/2と設定している。
#  (7) 何というデータセットを使い、先行研究に比べどういった指標値が得られているか。
MS COCOやPasacalというデータセットからmAPと特にfpsで優れた指標が出ている。